In [1]:
!pip install tensorflow==2.2.0

     |████████████████████████████████| 516.2MB 31kB/s 
     |████████████████████████████████| 460kB 39.9MB/s 
     |████████████████████████████████| 3.0MB 49.7MB/s 
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [2]:
import tensorflow as tf

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
#from tensorflow.keras.optimizers import Adam
from tensorflow import keras

import zipfile

In [3]:
print(tf.__version__)

2.2.0


In [4]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/vid_frame.zip", 'r')
zip_ref.extractall("/content/emotion")
zip_ref.close()

In [5]:
TRAINING_DIR = "/content/emotion/training/"
VALIDATION_DIR = "/content/emotion/validation/"

In [6]:
opt = keras.optimizers.Adam(learning_rate =0.001)
es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=30)
mc = ModelCheckpoint('/content/drive/My Drive/BestMobileNetV2.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

img_height = 224
img_width = 224
batch_size = 16
nb_epochs = 50

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255)
# validation_split=0.1) # set validation split

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')
# subset='training') # set as training data

val_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = train_datagen.flow_from_directory(
    VALIDATION_DIR, # same directory as training data
    target_size= (img_height, img_width),
    batch_size= batch_size,
    class_mode='categorical')
# subset='validation') # set as validation data

Found 52314 images belonging to 8 classes.
Found 5550 images belonging to 8 classes.


In [8]:
base_model = MobileNetV2(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
emotions = Dense(8, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=emotions)

for layer in base_model.layers:
  layer.trainable = True

9412608/9406464 [==============================] - 0s 0us/step


In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, None, None, 3 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, None, None, 3 128         Conv1[0][0]                      
______________________________________________________________________________________________

In [10]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])

model.fit_generator(train_generator,
                    steps_per_epoch = train_generator.samples // batch_size,
                    validation_data = validation_generator,
                    validation_steps = validation_generator.samples // batch_size,
                    epochs = nb_epochs, callbacks=[es, mc])

model.save('/content/drive/My Drive/MobileNetV2')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
3269/3269 [==============================] - ETA: 0s - loss: 0.6977 - acc: 0.7595
Epoch 00001: val_acc improved from -inf to 0.27890, saving model to /content/drive/My Drive/BestMobileNetV2.h5
3269/3269 [==============================] - 371s 113ms/step - loss: 0.6977 - acc: 0.7595 - val_loss: 8.4512 - val_acc: 0.2789
Epoch 2/50
3269/3269 [==============================] - ETA: 0s - loss: 0.3656 - acc: 0.8754
Epoch 00002: val_acc did not improve from 0.27890
3269/3269 [==============================] - 371s 113ms/step - loss: 0.3656 - acc: 0.8754 - val_loss: 8.4628 - val_acc: 0.2567
Epoch 3/50
3269/3269 [==============================] - ETA: 0s - loss: 0.2548 - acc: 0.9135
Epoch 00003: val_acc improved from 0.27890 to 0.29064, saving model to /content/drive/My Drive/BestMobileNetV2.h5
3269/3269 [==============================] - 369s 113ms/step - loss: 0.2548 - acc: 0.9135 - val_loss: 4.7746 - val_a

In [12]:
read_model = keras.models.load_model("/content/drive/My Drive/BestMobileNetV2.h5")

In [15]:
TESTING_DIR = "/content/emotion/testing/"

test_datagen = ImageDataGenerator(rescale=1./255)

testing_generator = test_datagen.flow_from_directory(
    TESTING_DIR, # same directory as training data
    target_size= (img_height, img_width),
    batch_size= batch_size,
    class_mode='categorical')

read_model.evaluate(testing_generator)

Found 5528 images belonging to 8 classes.
346/346 [==============================] - 12s 34ms/step - loss: 3.1125 - acc: 0.5308


[3.1124701499938965, 0.5307525396347046]